In [ ]:
%%bash
# Check nvcc version
nvcc -V
echo
# Check GCC version
gcc --version
echo
# Check the version of torch and cuda packages
pip list | grep "torch\|cuda"

In [ ]:
!pip install mmcv-full==1.3.8 -f https://download.openmmlab.com/mmcv/dist/cu110/torch1.7.0/index.html

In [ ]:
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
!cd mmdetection && pip install -e .

!pip install Pillow==7.0.0

In [ ]:
!pip install wandb --upgrade

In [ ]:

# I have saved my API token with "wandb_api" as Label. 
# If you use some other Label make sure to change the same below. 


import sys
sys.path.insert(0, "./mmdetection")

import os
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

# Check MMDetection installation

# Imports
import mmdet
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector

import random
import numpy as np
from pathlib import Path

In [ ]:
!cat mmdetection/configs/cascade_rcnn/README.md

In [ ]:
!mkdir checkpoints
!wget -c https://download.openmmlab.com/mmdetection/v2.0/cascade_rcnn/cascade_mask_rcnn_r50_caffe_fpn_mstrain_3x_coco/cascade_mask_rcnn_r50_caffe_fpn_mstrain_3x_coco_20210707_002651-6e29b3a6.pth -o checkpoints/cascade_mask_rcnn_r50_caffe_fpn_mstrain_3x_coco_20210707_002651-6e29b3a6.pth

In [ ]:
!ls checkpoints

In [ ]:
from mmcv import Config
cfg = Config.fromfile('mmdetection/configs/cascade_rcnn/cascade_rcnn_x101_64x4d_fpn_20e_coco.py')
cfg.dataset_type = 'CocoDataset'
cfg.classes = ("Covid_Abnormality",)
for head in cfg.model.roi_head.bbox_head:
    head.num_classes = 1

In [ ]:
job_folder='working_dir'
if not os.path.exists(job_folder):
    os.makedirs(job_folder)
cfg.dataset_type = 'CocoDataset'
cfg.data.train.type = 'CocoDataset'
cfg.data.train.img_prefix = 'train'
cfg.data.train.classes = cfg.classes
cfg.data.train.ann_file = '../input/covid-json-dataset/json/coco_768x768/train_annotations_fold0_768x768.json'
cfg.work_dir = job_folder
cfg.data.val.img_prefix = 'train' # Prefix of image path
cfg.data.val.classes = cfg.classes
cfg.data.val.ann_file = f'../input/covid-json-dataset/json/coco_768x768/val_annotations_fold0_768x768.json'
cfg.data.val.type='CocoDataset'

cfg.data.test.img_prefix = 'train' # Prefix of image path
cfg.data.test.classes = cfg.classes
cfg.data.test.ann_file = f'../input/covid-json-dataset/json/coco_768x768/val_annotations_fold0_768x768.json'
cfg.data.test.type='CocoDataset'

In [ ]:

# 
cfg.evaluation.metric = 'bbox' 


# Set the epoch intervel to perform evaluation
cfg.evaluation.interval = 1

# Set the iou threshold of the mAP calculation during evaluation
cfg.evaluation.iou_thrs = [0.5]

In [ ]:
cfg.gpu_ids = range(1)


In [ ]:
import json
import pandas as pd
import numpy as np


In [ ]:
meta = pd.read_csv('../input/covid-kaushal/meta_data.csv')
train_comb = pd.read_csv('../input/covid-kaushal/train_comb.csv')

In [ ]:
box_df = pd.read_csv('../input/covid-kaushal/new_train_boxes (1).csv')

In [ ]:
!mkdir train
!mkdir train
!tar -xf ../input/jpg-data/train.tar.gz -C train

In [ ]:
cfg.work_dir='train'


In [ ]:
cfg.runner.max_epochs = 12
cfg.total_epochs = 12
cfg.optimizer.lr = 0.02 / 8
cfg.lr_config.warmup = None
cfg.log_config.interval = 600

In [ ]:
model = build_detector(cfg.model)
datasets = [build_dataset(cfg.data.train)]
model.init_weights()

In [ ]:
cfg.runner.max_epochs = 12
cfg.total_epochs = 12
cfg.seed = 0

cfg.gpu_ids = range(1)

In [ ]:
cfg.optimizer.lr = 0.02/8

## Learning rate scheduler config used to register LrUpdater hook
cfg.lr_config = dict(
    policy='CosineAnnealing', # The policy of scheduler, also support CosineAnnealing, Cyclic, etc. Refer to details of supported LrUpdater from https://github.com/open-mmlab/mmcv/blob/master/mmcv/runner/hooks/lr_updater.py#L9.
    by_epoch=False,
    warmup='linear', # The warmup policy, also support `exp` and `constant`.
    warmup_iters=500, # The number of iterations for warmup
    warmup_ratio=0.001, # The ratio of the starting learning rate used for warmup
    min_lr=1e-07)

# config to register logger hook
cfg.log_config.interval = 20 # Interval to print the log

# Config to set the checkpoint hook, Refer to https://github.com/open-mmlab/mmcv/blob/master/mmcv/runner/hooks/checkpoint.py for implementation.
cfg.checkpoint_config.interval = 1
cfg.data.samples_per_gpu = 4 # Batch size of a single GPU used in testing
cfg.data.workers_per_gpu = 2
train_detector(model, datasets[0], cfg, distributed=False, validate=True)